In [12]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader
from torch import cuda, bfloat16
import transformers
from transformers import StoppingCriteria, StoppingCriteriaList
import torch
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import JSONLoader
from langchain.retrievers import TFIDFRetriever, EnsembleRetriever
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_transformers import (
    EmbeddingsRedundantFilter,
    EmbeddingsClusteringFilter,
)
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.document_transformers import LongContextReorder
from langchain.retrievers.document_compressors import LLMChainFilter
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever



In [106]:
!pip install pinecone-client openai tiktoken langchain
# !pip install faiss-cpu
# !pip install underthesea


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
def load_json_vectorstore():
    txt_loader = JSONLoader(
        file_path="/Users/macbook/mta/serving-chat/file.json",
        jq_schema=".[].text",
    )
    documents = txt_loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512, chunk_overlap=0,
    )
    
    all_splits = text_splitter.split_documents(documents)
    return all_splits

In [ ]:
import json
from underthesea import text_normalize
import pandas as pd
from langchain.schema import Document

df = pd.read_csv("c3.csv")
output_chunk = []
for doc in df.iterrows():
    # print(doc[1].text)
    output_chunk.append(Document(page_content=text_normalize(doc[1].text)))

In [3]:
# import pinecone
# from langchain.vectorstores import Pinecone

# # initialize pinecone
# pinecone.init(
#     api_key="288540e9-e7dc-4a9b-b4f8-383fe463ee87",  # find at app.pinecone.io
#     environment="gcp-starter",  # next to api key in console
# )

# index_name = "langchain-demo"

# # First, check if our index already exists. If it doesn't, we create it
# if index_name not in pinecone.list_indexes():
#     # we create a new index
#     pinecone.create_index(
#       name=index_name,
#       metric='cosine',
#       dimension=1024
# )
# # The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
# docsearch = Pinecone.from_documents(all_splits, embeddings, index_name=index_name)

# # if you already have an index, you can load it like this
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)



In [73]:
# all_splits = load_vectorstore()



all_splits = output_chunk
embeddings = HuggingFaceEmbeddings(model_name="vinai/phobert-base-v2")
vectorstore = FAISS.from_documents(all_splits, embeddings)
tfidf_retriever = TFIDFRetriever.from_documents(all_splits, search_type="similarity", search_kwargs={"k": 2, "include_metadata": False})
emb_retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2, "include_metadata": False})
qa_embeddings = HuggingFaceEmbeddings(model_name="vinai/bartpho-syllable")
_qavectorstore = FAISS.from_documents(all_splits, qa_embeddings)
qa_retriever = _qavectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2, "include_metadata": False})
ensemble_retriever = EnsembleRetriever(retrievers=[tfidf_retriever, emb_retriever, qa_retriever], weights=[0.4, 0.3, 0.2], search_kwargs={"k": 3, "include_metadata": False})


# merge_retriever = MergerRetriever(retrievers=[tfidf_retriever, emb_retriever])
# _filter = LLMChainFilter.from_llm(llm)

# _filter = EmbeddingsRedundantFilter(embeddings=filter_embeddings)
# reordering = LongContextReorder()
# pipeline = DocumentCompressorPipeline(transformers=[_filter, reordering])
# compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline, base_retriever=merge_retriever)
# The storage layer for the parent documents
# store = InMemoryStore()
# child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# parent_retriever = ParentDocumentRetriever(
#     vectorstore=_qavectorstore, 
#     docstore=store, 
#     child_splitter=child_splitter,
# )
# txt_loader = JSONLoader(
#         file_path="/Users/macbook/mta/serving-chat/file.json",
#         jq_schema=".[].text",
#         # content_key="text",
#         # metadata_func=metadata_func
#     )
# documents = txt_loader.load()
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=512, chunk_overlap=0,
# )

# all_splits = text_splitter.split_documents(documents)
# retriever = ParentDocumentRetriever(
#     vectorstore=vectorstore, 
#     docstore=store, 
#     child_splitter=child_splitter,
#     parent_splitter=parent_splitter,
# )


No sentence-transformers model found with name /Users/macbook/.cache/torch/sentence_transformers/vinai_phobert-base-v2. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/macbook/.cache/torch/sentence_transformers/vinai_phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
No sentence-transformers model found with name /Users/macbook/.cache/torch/sentence_transformers/vinai_bartpho-syllable. Creating a new one with MEAN pooling.


In [74]:
ensemble_retriever.get_relevant_documents("Giờ giảng dạy là gì?")

[Document(page_content='Điều 3 . Giờ hành chính và giờ chuẩn giảng dạy . Giờ hành chính : là đơn vị thời gian làm việc theo chế độ tuần làm việc 40 giờ được quy định trong Bộ luật Lao động ( 01 giờ hành chính bằng 60 phút ) . Giờ chuẩn giảng dạy : ( sau đây gọi tắt là giờ chuẩn ) là đơn vị thời gian quy đổi để thực hiện một công việc tương đương cho một tiết ( 45 phút ) giảng dạy lý thuyết trình độ đại học trực tiếp trên giảng đường hoặc giảng dạy trực tuyến , bao gồm thời gian lao động cần thiết trước , trong và sau tiết giảng .', metadata={}),
 Document(page_content='Quy đổi giờ chuẩn đối với hoạt động ngoại khóa . . Giảng viên chuyên trách kiêm nhiệm cố vấn học tập đơn vị tính 1 lớp tương ứng với 70 giờ chuẩn . Tham gia Hội đồng chấm thi phương pháp giảng dạy , Hội đồng thi tuyển giảng viên đơn vị tính 1 giảng viên tương ứng với 3 giờ chuẩn .', metadata={}),
 Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu - Tổng số giảng viên của khoa ( viện ) là số giảng viên đa

In [75]:

# print(len(chunk.page_content.split()))
ensemble_retriever.get_relevant_documents("Giờ chuẩn giảng dạy là gì?")

[Document(page_content='Điều 3 . Giờ hành chính và giờ chuẩn giảng dạy . Giờ hành chính : là đơn vị thời gian làm việc theo chế độ tuần làm việc 40 giờ được quy định trong Bộ luật Lao động ( 01 giờ hành chính bằng 60 phút ) . Giờ chuẩn giảng dạy : ( sau đây gọi tắt là giờ chuẩn ) là đơn vị thời gian quy đổi để thực hiện một công việc tương đương cho một tiết ( 45 phút ) giảng dạy lý thuyết trình độ đại học trực tiếp trên giảng đường hoặc giảng dạy trực tuyến , bao gồm thời gian lao động cần thiết trước , trong và sau tiết giảng .', metadata={}),
 Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu - Tổng số giảng viên của khoa ( viện ) là số giảng viên đang tham gia giảng dạy tại đơn vị trong năm học ( tính cả trợ giảng và giảng viên kiêm nhiệm ; không tính giảng viên thỉnh giảng ) .', metadata={}),
 Document(page_content='Điều 7 . Định mức giờ chuẩn đối với giảng viên . Số giờ chuẩn giảng dạy trực tiếp trên lớp hoặc giảng dạy trực tuyến của giảng viên giữ chức vụ lãnh đ

In [76]:
ensemble_retriever.get_relevant_documents("Giảng viên kiêm nhiệm là gì?")

[Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu - Tổng số giảng viên của khoa ( viện ) là số giảng viên đang tham gia giảng dạy tại đơn vị trong năm học ( tính cả trợ giảng và giảng viên kiêm nhiệm ; không tính giảng viên thỉnh giảng ) .', metadata={}),
 Document(page_content='Điều 7 . Định mức giờ chuẩn đối với giảng viên giữ chức vụ lãnh đạo , chỉ huy , quản lý và giảng viên kiêm nhiệm . Giảng viên được bổ nhiệm giữ chức vụ lãnh đạo , quản lý hoặc kiêm nhiệm công tác khác có trách nhiệm giảng dạy với định mức được tính theo tỷ lệ % của định mức giờ chuẩn tại Điều 5 ( Phụ lục 1 ) . Giảng viên kiêm nhiệm nhiều chức vụ thì áp dụng định mức giờ chuẩn có tỷ lệ yêu cầu nhỏ nhất quy định tại Khoản 1 Điều này .', metadata={}),
 Document(page_content='Điều 11 . Chế độ làm việc đối với giảng viên kiêm nghiệm .. Khoa , bộ môn phân công giảng dạy và tạo điều kiện để giảng viên kiêm nhiệm thực hiện tốt nhiệm vụ giảng dạy và hoạt động KHCN. Cơ quan nơi giảng viên kiêm nhiệm côn

In [77]:
ensemble_retriever.get_relevant_documents("Giảng viên thỉnh giảng là gì?")

[Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu - Tổng số giảng viên của khoa ( viện ) là số giảng viên đang tham gia giảng dạy tại đơn vị trong năm học ( tính cả trợ giảng và giảng viên kiêm nhiệm ; không tính giảng viên thỉnh giảng ) .', metadata={}),
 Document(page_content='Điều 12 . Chế độ làm việc đối với giảng viên thỉnh giảng . Thực hiện theo quy định hiện hành về mời giảng tại Học viện Kỹ thuật quân sự .', metadata={}),
 Document(page_content='Quy đổi giờ chuẩn đối với hoạt động ngoại khóa . . Giảng viên chuyên trách kiêm nhiệm cố vấn học tập đơn vị tính 1 lớp tương ứng với 70 giờ chuẩn . Tham gia Hội đồng chấm thi phương pháp giảng dạy , Hội đồng thi tuyển giảng viên đơn vị tính 1 giảng viên tương ứng với 3 giờ chuẩn .', metadata={}),
 Document(page_content='Điều 2 . Giảng viên tại Học viện Kỹ thuật quân sự . Giảng viên kiêm nhiệm : là những cán bộ trong Học viện không thuộc biên chế của các khoa , có tham gia công tác đào tạo . Giảng viên thỉnh giảng : là 

In [78]:
ensemble_retriever.get_relevant_documents("Quy đổi giờ chuẩn ngoại ngữ là bao nhiêu?")

[Document(page_content='Quy đổi giờ chuẩn đối với hoạt động hoạt động đào tạo có sử dụng ngoại ngữ . Thẩm định ngôn ngữ của ĐATN LVCH LATS viết bằng ngoại ngữ có 3 trường hợp : Đại học đơn vị tính 1 ĐATN tướng ứng 7 giờ chuẩn . Thạc sỹ đơn vị tính 1 LVCH tướng ứng 10 giờ chuẩn . Tiến sĩ đơn vị tính 1 LATS tương ứng với 15 giờ chuẩn . Hoạt động đào tạo bằng ngoại ngữ có 3 Trường hợp Dạy 100 % bằng ngoại ngữ đối với học phần không phải là ngoại ngữ đơn vị tính 1 tiết tướng ứng 2 giờ chuẩn .', metadata={}),
 Document(page_content='Quy đổi giờ chuẩn đối với hoạt động Dạy song ngữ đơn vị tính1 tiết tương ứng với 1.5 giờ chuẩn Các hoạt động đào tạo khác mà ngôn ngữ sử dụng là ngoại ngữ thì được tính tải bằng hệ số so với sử dụng Tiếng Việt tương ứng như sau : + Đối với đào tạo đại học : hệ số 13 + Đối với đào tạo sau đại học : hệ số 15', metadata={}),
 Document(page_content='Quy đổi giờ chuẩn đối với hoạt động 18 . Đối với nội dung công việc Tham gia hoạt động Tư liệu - Thư viện thì có 4 trư

In [79]:
ensemble_retriever.get_relevant_documents("Quy đổi giờ chuẩn đối với giám đốc là bao nhiêu?")

[Document(page_content='Quy đổi giờ chuẩn đối với hoạt động Thư ký đơn vị tính 1 CT tướng ứng 20 giờ chuẩn . Ủy viên đơn vị tính1 CT tướng ứng 15 giờ chuẩn .', metadata={}),
 Document(page_content='Tỷ lệ định mức giờ chuẩn giảng dạy đối với giảng viên giữ chức vụ chỉ huy , quản lý và giảm viên kiêm nhiệm . Với đối tượng Giám đốc , Chính ủy : Có tỉ lệ định mức giờ chuẩn là 10 % . Với đối tượng Phó Giám đốc , Phó Chính ủy : Có tỉ lệ định mức giờ chuẩn là 15 % . Với cán bộ quản lý , chỉ huy công tác đoàn thể : - Trưởng phòng ( trực thuộc Học viện ) và tương đương Có tỉ lệ định mức giờ chuẩn là 20 % . - Phó trưởng phòng ( trực thuộc Học viện ) , trưởng ban và tương đương Có tỉ lệ định mức giờ chuẩn là 25 % .', metadata={}),
 Document(page_content='Quy đổi giờ hành chính đối với hoạt động Khoa học công nghệ khác . Tham gia Hội đồng xét duyệt , tuyển chọn nhiệm vụ KHCN , Có 2 trường hợp : - nếu là Đề tài cấp Quốc gia thì đơn vị tính là 1 ĐT tương ứng với số giờ chuẩn là 9 giờ ; - nếu là Đề t

In [80]:
ensemble_retriever.get_relevant_documents("Quy định áp dụng với ai?")

[Document(page_content='Điều 1 . Phạm vi điều chỉnh và đối tượng áp dụng của quy định 2053 : Văn bản này quy định về chế độ làm việc của giảng viên tại Học viện Kỹ thuật quân sự ( sau đây gọi tắt là Học viện ) , bao gồm : Định mức thời gian làm việc , giờ chuẩn , chế độ làm việc và tính tải giảng dạy , tải hoạt động khoa học công nghệ ( KHCN ) . Đối tượng áp dụng : Quy định này được áp dụng cho toàn bộ đội ngũ giảng viên của Học viện .', metadata={}),
 Document(page_content='Điều 7 . Định mức giờ chuẩn đối với giảng viên giữ chức vụ lãnh đạo , chỉ huy , quản lý và giảng viên kiêm nhiệm . Giảng viên được bổ nhiệm giữ chức vụ lãnh đạo , quản lý hoặc kiêm nhiệm công tác khác có trách nhiệm giảng dạy với định mức được tính theo tỷ lệ % của định mức giờ chuẩn tại Điều 5 ( Phụ lục 1 ) . Giảng viên kiêm nhiệm nhiều chức vụ thì áp dụng định mức giờ chuẩn có tỷ lệ yêu cầu nhỏ nhất quy định tại Khoản 1 Điều này .', metadata={}),
 Document(page_content='Điều 16 . Quản lý , sử dụng thời gian làm v

In [81]:
ensemble_retriever.get_relevant_documents("Với tiêu chuẩn cán bộ quản lý học viên tốt, có giới hạn số lượng cán bộ quản lý học viên không quá bao nhiêu phần trăm tổng số của đơn vị??")

[Document(page_content='Tiêu chuẩn cán bộ quản lý học viên tốt . Đối tượng : Cán bộ tham gia quản lý học viên ( không xét đối với các trợ lý của các đơn vị quản lý học viên ) . Số lượng : Không quá 15 % tổng số cán bộ quản lý học viên của đơn vị . Tiêu chuẩn : Cán bộ quản lý học viên tốt là người gương mẫu trong việc thực hiện các quy chế liên quan đến giáo dục , đào tạo , quản lý học viên trong Học viện ;', metadata={}),
 Document(page_content='Tiêu chuẩn cán bộ quản lý học viên tốt . là người tiêu biểu về phẩm chất đạo đức , tác phong , năng lực quản lý , chỉ huy của người cán bộ giáo dục được đồng nghiệp và học viên yêu mến , tín nhiệm . - Cán bộ quản lý học viên tốt phải hoàn thành tốt các chế độ , chức trách theo quy định chung và quy định trong quy chế về công tác quản lý học viên tại Học viện , tham gia nghiên cứu về khoa học quản lý giáo dục ; năng động , tìm tòi , đề xuất sáng kiến để công tác quản lý giáo dục có hiệu quả , chất lượng cao . - Đạt chuẩn trình độ ngoại ngữ theo 

In [82]:
ensemble_retriever.get_relevant_documents("Theo tiêu chuẩn cán bộ quản lý học viên tốt, cán bộ quản lý học viên cần tham gia vào hoạt động nào để nâng cao hiệu quả và chất lượng công tác quản lý giáo dục??")

[Document(page_content='Tiêu chuẩn cán bộ quản lý học viên tốt . Đối tượng : Cán bộ tham gia quản lý học viên ( không xét đối với các trợ lý của các đơn vị quản lý học viên ) . Số lượng : Không quá 15 % tổng số cán bộ quản lý học viên của đơn vị . Tiêu chuẩn : Cán bộ quản lý học viên tốt là người gương mẫu trong việc thực hiện các quy chế liên quan đến giáo dục , đào tạo , quản lý học viên trong Học viện ;', metadata={}),
 Document(page_content='Tiêu chuẩn cán bộ quản lý học viên tốt . là người tiêu biểu về phẩm chất đạo đức , tác phong , năng lực quản lý , chỉ huy của người cán bộ giáo dục được đồng nghiệp và học viên yêu mến , tín nhiệm . - Cán bộ quản lý học viên tốt phải hoàn thành tốt các chế độ , chức trách theo quy định chung và quy định trong quy chế về công tác quản lý học viên tại Học viện , tham gia nghiên cứu về khoa học quản lý giáo dục ; năng động , tìm tòi , đề xuất sáng kiến để công tác quản lý giáo dục có hiệu quả , chất lượng cao . - Đạt chuẩn trình độ ngoại ngữ theo 

In [83]:
ensemble_retriever.get_relevant_documents("Để được xem là giảng viên dạy tốt, giảng viên cần đáp ứng những yêu cầu gì cụ thể???")


[Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu trong năm học , được đánh giá xếp loại cán bộ hoàn thành tốt nhiệm vụ trở lên . - Giảng viên dạy giỏi phải đạt ít nhất 90 % số phiếu tín nhiệm của giảng viên trong bộ môn , khoa và ít nhất 2/3 số phiếu tín nhiệm của Hội đồng xét giảng viên dạy giỏi , dạy tốt cấp Học viện . Số lượng các danh hiệu : Giảng viên dạy giỏi : không quá 15 % tổng số giảng viên của khoa ( viện ) . - Giảng viên dạy tốt : không quá 10 % tổng số giảng viên của khoa ( viện ) .', metadata={}),
 Document(page_content='Đối tượng xét giảng viên giỏi , tốt yêu cầu - Tổng số giảng viên của khoa ( viện ) là số giảng viên đang tham gia giảng dạy tại đơn vị trong năm học ( tính cả trợ giảng và giảng viên kiêm nhiệm ; không tính giảng viên thỉnh giảng ) .', metadata={}),
 Document(page_content='Tiêu chuẩn cụ thể đối với giảng viên dạy giỏi bao gồm giảng dạy : - Kết quả học tập của lớp học viên do giảng viên được phân công giảng dạy học phần hoặc môn học tron

In [152]:

OUTPUT_CHUNK = []
for chunk in all_splits:
    OUTPUT.append({ "text": chunk.page_content, "len": len(chunk.page_content.split())})
    # if len(chunk.page_content.split()) < 30:
    #     print(chunk.page_content)
    # # print(len(chunk.page_content.split()))
# df =pd.DataFrame(OUTPUT)
# df.to_csv("chunk_split.csv", index=False)

In [18]:
import json
from underthesea import text_normalize
import pandas as pd
from langchain.schema import Document

with open("/Users/macbook/mta/serving-chat/QD2053_2.json", "r") as f:
    json_data = json.load(f)

output_data = []
for item in json_data["document"][0]['subdocument']:
    for content1 in item['content']:
        for content2 in content1['content']:
            output_data.append({ "text": text_normalize(" ".join(content2['title'].replace('\n', '').split()).strip())})
output_chunk = []
for doc in output_data:
    output_chunk.append(Document(page_content=doc['text']))
    


# retriever = TFIDFRetriever.from_documents(
#     [
#         Document(page_content="foo"),
#         Document(page_content="bar"),
#         Document(page_content="world"),
#         Document(page_content="hello"),
#         Document(page_content="foo bar"),
#     ]
# )


# df =pd.DataFrame(output_data)
# df.to_csv("file.csv", index=False)
# with open("file.json", "w+") as f:
#     json.dump(output_data, f)


In [19]:
output_chunk[0].page_content

'Điều 1 . Phạm vi điều chỉnh và đối tượng áp dụng của quy định 2053 : Văn bản này quy định về chế độ làm việc của giảng viên tại Học viện Kỹ thuật quân sự ( sau đây gọi tắt là Học viện ) , bao gồm : Định mức thời gian làm việc , giờ chuẩn , chế độ làm việc và tính tải giảng dạy , tải hoạt động khoa học công nghệ ( KHCN ) . Đối tượng áp dụng : Quy định này được áp dụng cho toàn bộ đội ngũ giảng viên của Học viện .'

In [29]:
for chunk in all_splits:
    if len(chunk.page_content.split()) < 20:
        print(chunk.page_content)
    # print(len(chunk.page_content.split()))

hoặc các nhiệm vụ đặc thù: 320. - Tổng số: 1760.
hoàn thành nhiệm vụ KHCN (cho dù vẫn đạt định mức tải KHCN theo yêu cầu).
tính 5,0 giờ chuẩn. Trường hợp còn lại, một ngày hướng dẫn được tính 2,5 giờ chuẩn.
học cấp Học viện được đánh giá, nghiệm thu Đạt trở lên.
viện được đánh giá, nghiệm thu đạt trở lên.
với số giờ chuẩn là 90;


In [ ]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_chain_type(llm = llm,
                                       chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
                                       retriever = retriever, 
                                       chain_type_kwargs = {"prompt": PROMPT},
                                       return_source_documents = True,
                                       verbose = False)